# EKE preprocessing

This notebook shows how to generate the EKE_timeseries.nc file

In [1]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [2]:
from dask.distributed import Client

In [3]:
from utils import area,ccrs_land,add_patches
import datetime

In [4]:
c = Client()
c

Client Scheduler: tcp://127.0.0.1:33109 Dashboard: /proxy/8787/status,Cluster Workers: 6 Cores: 36 Memory: 154.62 GB


In [5]:
files = '/g/data/ua8/CMEMS_SeaLevel/v2019/*/*.nc'
# files='path2AVISO+/*.nc'

In [6]:
dataset_AVISO = xr.open_mfdataset(files,parallel=True)

In [7]:
dataset_AVISO

<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, nv: 2, time: 9928)
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2020-03-07
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * nv         (nv) int32 0 1
Data variables:
    crs        (time) int32 -2147483647 -2147483647 ... -2147483647 -2147483647
    lat_bnds   (time, latitude, nv) float32 dask.array<chunksize=(1, 720, 2), meta=np.ndarray>
    lon_bnds   (time, longitude, nv) float32 dask.array<chunksize=(1, 1440, 2), meta=np.ndarray>
    err        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes:
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    creator_name:                    CMEMS - Sea Level Thematic Assembly Center
    creator_url:                     http://marine.copernicus.eu
    date_created:                    2019-01-17T16:58:50Z
    date_issued:                     2019-01-17T16:58:50Z
    date_modified:                   2019-01-17T16:58:50Z
    geospatial_lat_max:              89.875
    geospatial_lat_min:              -89.875
    geospatial_lat_resolution:       0.25
    geospatial_lat_units:            degrees_north
    geospatial_lon_max:              359.875
    geospatial_lon_min:              0.125
    geospatial_lon_resolution:       0.25
    geospatial_lon_units:            degrees_east
    geospatial_vertical_max:         0.0
    geospatial_vertical_min:         0.0
    geospatial_vertical_positive:    down
    geospatial_vertical_resolution:  point
    geospatial_vertical_units:       m
    history:                         2019-01-17 16:58:51Z: Creation
    institution:                     CLS, CNES
    keywords:                        Oceans > Ocean Topography > Sea Surface ...
    keywords_vocabulary:             NetCDF COARDS Climate and Forecast Stand...
    license:                         http://marine.copernicus.eu/web/27-servi...
    platform:                        ERS-1, Topex/Poseidon,
    processing_level:                L4
    product_version:                 2019
    project:                         COPERNICUS MARINE ENVIRONMENT MONITORING...
    references:                      http://marine.copernicus.eu
    software_version:                6.2_DUACS_DT2018_baseline
    source:                          Altimetry measurements
    ssalto_duacs_comment:            The reference mission used for the altim...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-01T00:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1993-01-01T00:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...

### Compute EKE:

\begin{equation}
EKE = \frac{1}{2} \left(u'^2+v'^2\right)
\end{equation}

The provided $u'$ and $v'$ from AVISO is an anomaly from 1993-2005. Changing this period does not affect the main result of the study.

In [8]:
EKE = 1/2 * (dataset_AVISO.ugosa**2 + dataset_AVISO.vgosa**2)

Rechunk dataset

In [9]:
EKE_rechunk = EKE.chunk({"latitude": 100, "longitude": 100,'time':357})

Coarsen dataset to a 1 degree grid.

In [10]:
EKE_coarsen = EKE_rechunk.coarsen({'latitude':4,'longitude':4}).mean()

Roll over 12 months:

In [11]:
EKE_rolled = EKE_coarsen.rolling(time=365,center=True).mean().compute() 
# This computation requires ~ 50 GB of RAM, to decrease the usage of RAM, 
# reduce the size of the chunks.

In [12]:
EKE_rolled = EKE_rolled.to_dataset(name="EKE").rename({'latitude':'lat','longitude':'lon'})

In [13]:
EKE_rolled

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 9928)
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2020-03-07
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Data variables:
    EKE      (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan

### Add metadata

In [14]:
EKE_rolled.attrs['title'] = "Eddy Kinetic Energy"
EKE_rolled.attrs['Description'] = """Eddy Kinetic Energy computed from AVISO+ altimetry products."""
EKE_rolled.attrs['Publication'] = "Dataset created for Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
EKE_rolled.attrs['Author'] = "Josué Martínez-Moreno"
EKE_rolled.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

EKE_rolled.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

counter = 0
units = [r"$m^2s^{-2}$"]
names = ["EKE"]
long_names = ["Eddy Kinetic Energy"]

EKE_rolled['EKE'].attrs['units'] = units[counter]
EKE_rolled['EKE'].attrs['name'] = names[counter]
EKE_rolled['EKE'].attrs['long_name'] = long_names[counter]

EKE_rolled['EKE'].attrs['missing_value'] = np.nan
EKE_rolled['EKE'].attrs['valid_min'] = np.nanmin(EKE_rolled['EKE'])
EKE_rolled['EKE'].attrs['valid_max'] = np.nanmax(EKE_rolled['EKE'])
EKE_rolled['EKE'].attrs['valid_range'] = [np.nanmin(EKE_rolled['EKE']),np.nanmax(EKE_rolled['EKE'])]

## Store netCDF

In [15]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in EKE_rolled.data_vars}

EKE_rolled.to_netcdf('../datasets/EKE_timeseries.nc', encoding=encoding)